In [8]:
import pandas as pd

filtered_full_sample = pd.read_csv('data/filtered_full_sample.csv', index_col=0).drop(columns=['magic_COUR'])

In [9]:
filtered_full_sample.demo_gender = filtered_full_sample.demo_gender == "female"

In [20]:
country_demographics = filtered_full_sample.groupby("geo_country").mean()
country_demographics = country_demographics.merge(
    filtered_full_sample.groupby("geo_country").count(),
    left_index=True,
    right_index=True,
    suffixes=("_mean", "_count")
).merge(
    filtered_full_sample.groupby("geo_country").std(),
    left_index=True,
    right_index=True
)
country_demographics.columns = [
    "age_mean", "percent_female", "N", "n", "age_sd", "g"
]
country_demographics.age_mean = country_demographics.age_mean.round(2)
country_demographics.age_sd = country_demographics.age_sd.round(2)
country_demographics.percent_female = (country_demographics.percent_female * 100).round(1)
country_demographics["percent_of_full"] = (country_demographics.N / len(filtered_full_sample) * 100).round(2)

country_demographics = country_demographics[['N', 'percent_of_full', 'percent_female', 'age_mean', 'age_sd']]
country_demographics.to_csv('results/country_demographics.csv')
country_demographics

,N,percent_of_full,percent_female,age_mean,age_sd
geo_country,,,,,
Argentina,2139,0.14,74.8,22.14,7.10
Australia,30960,2.03,71.5,24.08,10.74
Austria,1804,0.12,72.5,23.41,9.10
Bangladesh,780,0.05,62.8,19.47,6.37
Belarus,316,0.02,80.1,23.31,6.40
...,...,...,...,...,...
United Kingdom,53047,3.48,68.6,24.66,10.71
United States,1147164,75.30,69.9,27.60,11.32
Uruguay,354,0.02,70.3,22.36,6.92
